# Client segmentation of the online present shop

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import warnings 

from IPython.display import display, HTML

warnings.filterwarnings("ignore")

plt.rcParams["patch.force_edgecolor"] = True

## 1. Data description

In [2]:
data = pd.read_csv(
    "data/customer_segmentation_project.csv", 
    encoding="ISO-8859-1", 
    dtype={'CustomerID': str,'InvoiceID': str}
)
print(f'Data shape: {data.shape}')
data.head()

Data shape: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  object 
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


Missing data per feature:

In [4]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Check datetime span in the data:

In [5]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
print(f"The first date in dataset: {data['InvoiceDate'].min()}")
print(f"The last date in dataset: {data['InvoiceDate'].max()}")

The first date in dataset: 2010-12-01 08:26:00
The last date in dataset: 2011-12-09 12:50:00


In [6]:
data.describe()

,Quantity,InvoiceDate,UnitPrice
count,541909.000000,541909,541909.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114
min,-80995.000000,2010-12-01 08:26:00,-11062.060000
25%,1.000000,2011-03-28 11:34:00,1.250000
50%,3.000000,2011-07-19 17:17:00,2.080000
75%,10.000000,2011-10-19 11:27:00,4.130000
max,80995.000000,2011-12-09 12:50:00,38970.000000
std,218.081158,NaN,96.759853


The cheapest unit with non-negative price:

In [7]:
data[data['UnitPrice'] >= 0]['UnitPrice'].min()

0.0

Number of unique clients:

In [8]:
data['CustomerID'].nunique()

4372

Number of unique country excluding 'Unspecified' code:

In [9]:
data[~(data['Country'] == 'Unspecified')]['Country'].nunique()

37

The most popular StockCode:

In [10]:
data['StockCode'].mode()

0    85123A
Name: StockCode, dtype: object

In [11]:
invoice_code = data['InvoiceNo'].apply(lambda x: x[0])
invoice_code.value_counts()

InvoiceNo
5    532618
C      9288
A         3
Name: count, dtype: int64

In [12]:
data[data['UnitPrice'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [13]:
data[data['Quantity'] < 0]['InvoiceNo'].apply(lambda x: x[0]).value_counts()

InvoiceNo
C    9288
5    1336
Name: count, dtype: int64

## 2. Data preprocessing and cleaning

In [14]:
print(f'Total number of missing data: {data.isna().sum().sum()}')

Total number of missing data: 136534


Delete missing data in `Description` and `CustomerID` features:

In [15]:
data.dropna(axis=0, inplace=True)
print(f'Number of remaining entries: {data.shape[0]}')

Number of remaining entries: 406829


Find duplicates:

In [16]:
print(f'Number of duplicated entries: {data.duplicated().sum()}')
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True, drop=True)
print(f'Number of remaining entries: {data.shape[0]}')

Number of duplicated entries: 5225
Number of remaining entries: 401604


Let's find out where negative quantity comes from:

In [17]:
negative_quantity = data[(data['Quantity'] < 0)].copy()
print(f'Count of entries with a negative number: {negative_quantity.shape[0]}')
negative_quantity.head()

Count of entries with a negative number: 8872


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom


Check whether all units with negative quantity are canceled:

In [18]:
negative_quantity['InvoiceNo'].apply(lambda x: x[0]).value_counts()

InvoiceNo
C    8872
Name: count, dtype: int64

Determine number of units per order:

In [19]:
temp = data.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()
nb_products_per_basket = temp.rename(columns={'InvoiceDate': 'Number of products'})
nb_products_per_basket.head()

,CustomerID,InvoiceNo,Number of products
0,12346,541431,1
1,12346,C541433,1
2,12347,537626,31
3,12347,542237,29
4,12347,549222,24


Determine how many orders were canceled:

In [20]:
nb_products_per_basket['order_canceled'] = nb_products_per_basket['InvoiceNo'].apply(lambda x: 1 if x[0] == 'C' else 0)
print(f"Percent of canceled orders: {np.round(nb_products_per_basket['order_canceled'].mean() * 100)}")
nb_products_per_basket.head()

Percent of canceled orders: 16.0


,CustomerID,InvoiceNo,Number of products,order_canceled
0,12346,541431,1,0
1,12346,C541433,1,1
2,12347,537626,31,0
3,12347,542237,29,0
4,12347,549222,24,0


In [21]:
negative_quantity_copy = negative_quantity.copy()
negative_quantity_copy['Quantity'] = -negative_quantity_copy['Quantity']
canceled_merged = negative_quantity_copy.merge(
    data,
    how='inner',
    on=['StockCode', 'Quantity', 'CustomerID']
)
canceled_merged.drop_duplicates(subset=['InvoiceNo_x', 'StockCode'], inplace=True)
canceled_merged.reset_index(inplace=True, drop=True)
print(f"Number of canceled orders without pair: {negative_quantity.shape[0] - canceled_merged.shape[0]}")
canceled_merged.head()

Number of canceled orders without pair: 5647


,InvoiceNo_x,StockCode,Description_x,Quantity,InvoiceDate_x,UnitPrice_x,CustomerID,Country_x,InvoiceNo_y,Description_y,InvoiceDate_y,UnitPrice_y,Country_y
0,C536543,22355,CHARLOTTE BAG SUKI DESIGN,2,2010-12-01 14:30:00,0.85,17841,United Kingdom,546543,CHARLOTTE BAG SUKI DESIGN,2011-03-14 15:40:00,0.85,United Kingdom
1,C536548,22168,ORGANISER WOOD ANTIQUE WHITE,2,2010-12-01 14:33:00,8.50,12472,Germany,542215,ORGANISER WOOD ANTIQUE WHITE,2011-01-26 12:27:00,8.50,Germany
2,C536622,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-02 10:37:00,8.50,12471,Germany,538174,SET 7 BABUSHKA NESTING BOXES,2010-12-10 09:35:00,8.50,Germany
3,C536734,22780,LIGHT GARLAND BUTTERFILES PINK,4,2010-12-02 12:50:00,4.25,16042,United Kingdom,540822,LIGHT GARLAND BUTTERFILES PINK,2011-01-11 13:25:00,4.25,United Kingdom
4,C538109,22780,LIGHT GARLAND BUTTERFILES PINK,4,2010-12-09 15:23:00,4.25,16042,United Kingdom,540822,LIGHT GARLAND BUTTERFILES PINK,2011-01-11 13:25:00,4.25,United Kingdom


Same hypothesis excluding discounts:

In [22]:
data_discount_off = data[~(data['StockCode'] == 'D')].copy()
neg_discount_off = data_discount_off[data_discount_off['Quantity'] < 0].copy()
neg_discount_off['Quantity'] = -neg_discount_off['Quantity']
canceled_discount_off = neg_discount_off.merge(
    data_discount_off,
    how='inner',
    on=['StockCode', 'Quantity', 'CustomerID']
)
canceled_discount_off.drop_duplicates(subset=['InvoiceNo_x', 'StockCode'], inplace=True)
canceled_discount_off.reset_index(inplace=True, drop=True)
print(f"Number of canceled orders without (discount excluded): {neg_discount_off.shape[0] - canceled_discount_off.shape[0]}")
canceled_discount_off.head()

Number of canceled orders without (discount excluded): 5570


,InvoiceNo_x,StockCode,Description_x,Quantity,InvoiceDate_x,UnitPrice_x,CustomerID,Country_x,InvoiceNo_y,Description_y,InvoiceDate_y,UnitPrice_y,Country_y
0,C536543,22355,CHARLOTTE BAG SUKI DESIGN,2,2010-12-01 14:30:00,0.85,17841,United Kingdom,546543,CHARLOTTE BAG SUKI DESIGN,2011-03-14 15:40:00,0.85,United Kingdom
1,C536548,22168,ORGANISER WOOD ANTIQUE WHITE,2,2010-12-01 14:33:00,8.50,12472,Germany,542215,ORGANISER WOOD ANTIQUE WHITE,2011-01-26 12:27:00,8.50,Germany
2,C536622,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-02 10:37:00,8.50,12471,Germany,538174,SET 7 BABUSHKA NESTING BOXES,2010-12-10 09:35:00,8.50,Germany
3,C536734,22780,LIGHT GARLAND BUTTERFILES PINK,4,2010-12-02 12:50:00,4.25,16042,United Kingdom,540822,LIGHT GARLAND BUTTERFILES PINK,2011-01-11 13:25:00,4.25,United Kingdom
4,C538109,22780,LIGHT GARLAND BUTTERFILES PINK,4,2010-12-09 15:23:00,4.25,16042,United Kingdom,540822,LIGHT GARLAND BUTTERFILES PINK,2011-01-11 13:25:00,4.25,United Kingdom


Duplicated orders:

In [23]:
orders_duplicated = data[data.duplicated(subset=data.drop(['Description', 'UnitPrice'], axis=1).columns)].copy()
print(f"Number of duplicated orders: {orders_duplicated.shape[0]}")
orders_duplicated.head()

Number of duplicated orders: 55


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1665,536569,M,Manual,1,2010-12-01 15:35:00,18.95,16274,United Kingdom
6173,537140,M,Manual,1,2010-12-05 12:53:00,0.85,12748,United Kingdom
11845,537804,M,Manual,12,2010-12-08 13:17:00,0.19,12748,United Kingdom
19092,538807,M,Manual,1,2010-12-14 12:06:00,5.00,17194,United Kingdom
20275,C538897,D,Discount,-1,2010-12-15 09:14:00,42.50,16422,United Kingdom


In [24]:
orders_duplicated['StockCode'].value_counts()

StockCode
M               32
D               12
POST             2
21181            1
21730            1
23298            1
22469            1
BANK CHARGES     1
23217            1
22273            1
23203            1
23407            1
Name: count, dtype: int64

Calculate quantity of canceled units with following exceptions:

In [25]:
def get_quantity_canceled(data):
    """Calculate quantity of canceled units

    Args:
        data (pandas.DataFrame): dataset to operate

    Returns:
        pandas.Series: column containing number of canceled products
    """
    
    # Initializing result array
    quantity_canceled = pd.Series(np.zeros(data.shape[0]), index=data.index)
    negative_quantity = data[(data['Quantity'] < 0)].copy()
    
    for index, col in negative_quantity.iterrows():
        # DataFrame with transactions which were canceled afterwise
        df_test = data[(data['CustomerID'] == col['CustomerID']) &
                       (data['StockCode']  == col['StockCode']) &
                       (data['InvoiceDate'] < col['InvoiceDate']) &
                       (data['Quantity'] > 0)].copy()
        
        # Canceled transaction has no counterpart - do nothing
        if (df_test.shape[0] == 0):
            continue
        # Canceled transaction has only one counterpart:
        # Add quantity of canceled products to quantity_canceled
        elif (df_test.shape[0] == 1):
            index_order = df_test.index[0]
            quantity_canceled.loc[index_order] = -col['Quantity']
        # Canceled transaction has several counterparts:
        # Each iteration add quantity of canceled products to quantity_canceled
        elif (df_test.shape[0] > 1):
            df_test.sort_index(axis=0 ,ascending=False, inplace=True)
            
            for ind, val in df_test.iterrows():
                # Quantity of canceled products is more than ordered ones - do nothing
                if val['Quantity'] < -col['Quantity']:
                    continue
                quantity_canceled.loc[ind] = -col['Quantity']
                break
    
    return quantity_canceled

In [26]:
data['QuantityCanceled'] = get_quantity_canceled(data)
print(f"Total number of canceled units: {data['QuantityCanceled'].sum()}")

Total number of canceled units: 245266.0


In [39]:
data.to_csv('data/data_temp.csv', index=False)

## Loading

In [3]:
data = pd.read_csv('./data/data_temp.csv')
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityCanceled
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,0.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,0.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0.0


Further we will work only with entries with non-negative quantity:

In [4]:
data_quantity_pos = data[~(data['Quantity'] < 0)].copy()
data_quantity_pos.shape

(392732, 9)

Special stock codes:

In [5]:
special_codes = [code for code in data_quantity_pos['StockCode'].unique() if code[0].isalpha()]
print(f"Number of special stock codes: {len(special_codes)}")

Number of special stock codes: 6


Drop entries with special stock codes:

In [6]:
data_digit_code = data_quantity_pos[~(data_quantity_pos['StockCode'].isin(special_codes))].copy()
print(f"Number of entries with digit only stock codes: {data_digit_code.shape[0]}")
data_digit_code.head()

Number of entries with digit only stock codes: 391183


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityCanceled
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,0.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,0.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0.0


Transactions with zero unit price:

In [7]:
print(f"Number of transactions with zero unit price: {data_digit_code[data_digit_code['UnitPrice'] == 0].shape[0]}")
print(f"Percent of transactions with zero unit price: {data_digit_code[data_digit_code['UnitPrice'] == 0].shape[0] / data_digit_code.shape[0] * 100:.2f}")
data_digit_code[data_digit_code['UnitPrice'] == 0].sample(5)

Number of transactions with zero unit price: 33
Percent of transactions with zero unit price: 0.01


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityCanceled
277443,569716,22778,GLASS CLOCHE SMALL,2,2011-10-06 08:17:00,0.0,15804,United Kingdom,0.0
25551,539722,22423,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,0.0,14911,EIRE,0.0
358000,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,2,2011-11-18 13:23:00,0.0,12444,Norway,0.0
374208,578841,84826,ASSTD DESIGN 3D PAPER STICKERS,12540,2011-11-25 15:57:00,0.0,13256,United Kingdom,0.0
29376,540372,22553,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.0,13081,United Kingdom,0.0


In [8]:
data_nonzero_price = data_digit_code[~(data_digit_code['UnitPrice'] == 0)].copy()
print(f"Number of nonzero transactions: {data_nonzero_price.shape[0]}")

Number of nonzero transactions: 391150


## 3. Exploratory Data Analysis

Figure out how many customers reside in each country:

In [9]:
country_client_num = data_nonzero_price.groupby(by='Country')['CustomerID'].count().sort_values(ascending=False)

country_clients_fig = px.bar(
    country_client_num,
    x=country_client_num.index,
    y=country_client_num.values
    )
country_clients_fig.update_layout(
        autosize=False, 
        width=1000, 
        height=600,
        title_text="Number of clients per country")
country_clients_fig.layout.xaxis.title.text = "Country"
country_clients_fig.layout.yaxis.title.text = "Number of clients"
country_clients_fig.show()

Figure out how many orders were from each country:

In [10]:
country_order_num = data_nonzero_price.groupby(by='Country')['InvoiceNo'].count().sort_values(ascending=False)

country_order_fig = px.bar(
    country_order_num,
    x=country_order_num.index,
    y=country_order_num.values
    )
country_order_fig.update_layout(
        autosize=False, 
        width=1000, 
        height=600,
        title_text="Number of orders per country")
country_order_fig.layout.xaxis.title.text = "Country"
country_order_fig.layout.yaxis.title.text = "Number of orders"
country_order_fig.show()

Add `TotalPrice` feature:

In [11]:
data_nonzero_price['TotalPrice'] = data_nonzero_price['UnitPrice'] * (data_nonzero_price['Quantity'] - \
    data_nonzero_price['QuantityCanceled'])
np.round(data_nonzero_price['TotalPrice'].mean())

21.0

Determine total income per country:

In [12]:
country_price_sum = data_nonzero_price.groupby(by='Country')['TotalPrice'].sum().sort_values(ascending=False)

country_price_fig = px.bar(
    country_price_sum,
    x=country_price_sum.index,
    y=country_price_sum.values
    )
country_price_fig.update_layout(
        autosize=False, 
        width=1000, 
        height=600,
        title_text="Total income per country")
country_price_fig.layout.xaxis.title.text = "Country"
country_price_fig.layout.yaxis.title.text = "Total income, pounds"
country_price_fig.show()

Generate datetime features:

In [13]:
data_nonzero_price['Month'] = data_nonzero_price['InvoiceDate'].dt.month
data_nonzero_price['Day_of_week'] = data_nonzero_price['InvoiceDate'].dt.weekday
data_nonzero_price['Hour'] = data_nonzero_price['InvoiceDate'].dt.hour
data_nonzero_price.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityCanceled,TotalPrice,Month,Day_of_week,Hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,0.0,15.30,12,2,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0.0,20.34,12,2,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,0.0,22.00,12,2,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0.0,20.34,12,2,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0.0,20.34,12,2,8


In [14]:
month_price_sum = data_nonzero_price.groupby(by='Month')['TotalPrice'].sum()

month_price_fig = px.bar(
    month_price_sum,
    x=month_price_sum.index,
    y=month_price_sum.values
    )
month_price_fig.update_layout(
        autosize=False, 
        width=1000, 
        height=600,
        title_text="Total income per month")
month_price_fig.layout.xaxis.title.text = "Month"
month_price_fig.layout.yaxis.title.text = "Total income, pounds"
month_price_fig.show()

In [15]:
dow_order_count = data_nonzero_price.groupby(by='Day_of_week')['InvoiceNo'].count()

dow_order_fig = px.bar(
    dow_order_count,
    x=dow_order_count.index,
    y=dow_order_count.values
    )
dow_order_fig.update_layout(
        autosize=False, 
        width=1000,
        height=600,
        title_text="Number of orders per day of week")
dow_order_fig.layout.xaxis.title.text = "Day of week"
dow_order_fig.layout.yaxis.title.text = "Number of orders"
dow_order_fig.show()

In [16]:
data_nonzero_price['Date'] = data_nonzero_price['InvoiceDate'].dt.date
order_date_pt = data_nonzero_price.pivot_table(
    values='InvoiceNo',
    index='Date',
    columns='Hour',
    aggfunc='count',
    fill_value=0
)
order_date_pt.head()

Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Date,,,,,,,,,,,,,,,
2010-12-01,0,0,45,149,114,351,412,256,174,183,159,49,0,0,0
2010-12-02,0,10,52,87,201,240,196,232,186,316,128,88,122,99,0
2010-12-03,0,0,0,36,104,111,220,72,246,127,55,105,0,0,0
2010-12-05,0,0,0,0,160,388,676,557,234,327,248,0,0,0,0
2010-12-06,0,0,31,42,115,250,585,295,274,224,17,56,0,0,0


In [17]:
order_date_mean = order_date_pt.mean()

hour_mean_fig = px.bar(
    order_date_mean,
    x=order_date_mean.index,
    y=order_date_mean.values
    )
hour_mean_fig.update_layout(
        autosize=False, 
        width=1000,
        height=600,
        title_text="Mean number of orders per hour")
hour_mean_fig.layout.xaxis.title.text = "Hour"
hour_mean_fig.layout.yaxis.title.text = "Mean number of orders"
hour_mean_fig.show()

## 4. RFM. PCA

In [18]:
RFM = pd.DataFrame()
t0 = pd.to_datetime('2011-12-10 00:00:00')
RFM['Recency'] = (t0 - data_nonzero_price.groupby(by='CustomerID')['InvoiceDate'].max()).dt.days
RFM['Frequency'] = data_nonzero_price.groupby(by='CustomerID')['InvoiceNo'].nunique()
RFM['Monetary'] = data_nonzero_price.groupby(by='CustomerID')['TotalPrice'].sum()
RFM.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346,325,1,0.00
12347,2,7,4310.00
12348,75,4,1437.24
12349,18,1,1457.55
12350,310,1,294.40


In [19]:
RFM[RFM['Recency'] > 200].shape[0]

743

In [20]:
np.round(RFM['Frequency'].mean())

4.0

In [21]:
np.round(RFM.loc[12360]['Monetary'])

2302.0

In [22]:
boxes = [px.box(RFM, x=column) for column in RFM.columns]

rfm_fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=(
        "Recency", "Frequency", "Monetary"
    )
)

for i, box in enumerate(boxes):
    rfm_fig.add_trace(boxes[i]['data'][0], row=1, col=i+1)

rfm_fig.update_layout(
    showlegend=True, 
    autosize=False, 
    width=1000,
    height=600)
rfm_fig.show()

In [23]:
RFM_filtered = RFM[(RFM['Frequency'] <= np.quantile(RFM['Frequency'], 0.95)) & \
    (RFM['Monetary'] <= np.quantile(RFM['Monetary'], 0.95))].copy()
RFM_filtered.shape[0]

4044

In [24]:
boxes_filtered = [px.box(RFM_filtered, x=column) for column in RFM_filtered.columns]

rfm_filtered_fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=(
        "Recency", "Frequency", "Monetary"
    )
)

for i, box in enumerate(boxes_filtered):
    rfm_filtered_fig.add_trace(boxes_filtered[i]['data'][0], row=1, col=i+1)

rfm_filtered_fig.update_layout(
    showlegend=True, 
    autosize=False, 
    width=1000,
    height=600)
rfm_filtered_fig.show()

In [25]:
rfm_3d_fig = px.scatter_3d(
    RFM_filtered,
    x='Recency',
    y='Frequency',
    z='Monetary'
)
rfm_3d_fig.update_traces(marker_size=3)
rfm_3d_fig.update_layout(
    showlegend=False, 
    autosize=False, 
    width=1000,
    height=600,
    title_text="RFM diagram")
rfm_3d_fig.show()

> This diagram shows nothing particular. The best option here would be dimension reduction.

In [26]:
pca_pipeline = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=2))])
rfm_pca = pca_pipeline.fit_transform(RFM_filtered)
print(f'Explained variance by the 1st component: {pca_pipeline[1].explained_variance_ratio_[0]:.2f}')

Explained variance by the 1st component: 0.68


In [27]:
rfm_pca_fig = px.scatter(
    rfm_pca,
    x=rfm_pca[:, 0],
    y=rfm_pca[:, 1]
)
rfm_pca_fig.update_traces(marker_size=3)
rfm_pca_fig.update_layout(
    showlegend=False, 
    autosize=False, 
    width=1000,
    height=600,
    title_text="RFM, 2 components PCA applied")
rfm_pca_fig.show()

> The density-based algorithms (such as DBSCAN) don't fit here because points are too close to each other which in its turn would result in one cluster.

### 4.1 Kmeans

In [28]:
silhouette_pca_kmeans_max = 0
silhouette_pca_kmeans_list = []
kmeans_pca_cluster_opt = 0

for cluster_num in range(2, 11):
    kmeans_pca_var = KMeans(n_clusters=cluster_num, random_state=42)
    kmeans_pca_var.fit(rfm_pca)
    silhouette_pca_kmeans_list.append(silhouette_score(rfm_pca, kmeans_pca_var.labels_))
    if silhouette_score(rfm_pca, kmeans_pca_var.labels_) > silhouette_pca_kmeans_max:
        silhouette_pca_kmeans_max = silhouette_score(rfm_pca, kmeans_pca_var.labels_)
        kmeans_pca_cluster_opt = cluster_num
    
print(f"Optimal number of clusters: {kmeans_pca_cluster_opt}")
print(f"Best silhouette score: {silhouette_pca_kmeans_max:.3f}")

Optimal number of clusters: 3
Best silhouette score: 0.524


In [29]:
kmeans_pca_silhouette_fig = px.line(
    x=range(2, 11),
    y=silhouette_pca_kmeans_list
)
kmeans_pca_silhouette_fig.update_layout(
        autosize=False, 
        width=1000,
        height=600,
        title_text="Kmeans, PCA, silhouette score vs number of clusters")
kmeans_pca_silhouette_fig.layout.xaxis.title.text = "Number of clusters"
kmeans_pca_silhouette_fig.layout.yaxis.title.text = "Silhouette score"
kmeans_pca_silhouette_fig.show()

In [31]:
kmeans_pca_opt = KMeans(n_clusters=kmeans_pca_cluster_opt, random_state=42)
kmeans_pca_opt.fit(rfm_pca)

rfm_pca_kmeans_fig = px.scatter(
    rfm_pca,
    x=rfm_pca[:, 0],
    y=rfm_pca[:, 1],
    color=kmeans_pca_opt.labels_
)
rfm_pca_kmeans_fig.update_traces(marker_size=3)
rfm_pca_kmeans_fig.update_layout(
    showlegend=False, 
    autosize=False, 
    width=1000,
    height=600,
    title_text=f"RFM, 2 components, Kmeans, PCA, {kmeans_pca_cluster_opt} clusters")
rfm_pca_kmeans_fig.show()

### 4.2 Gaussian mixture

In [32]:
silhouette_pca_GMM_max = 0
silhouette_pca_GMM_list = []
GMM_pca_cluster_opt = 0

for cluster_num in range(2, 11):
    GMM_pca_var = GaussianMixture(n_components=cluster_num, random_state=42)
    GMM_pca_var.fit(rfm_pca)
    GMM_pca_labels = GMM_pca_var.predict(rfm_pca)
    silhouette_pca_GMM_list.append(silhouette_score(rfm_pca, GMM_pca_labels))
    if silhouette_score(rfm_pca, GMM_pca_labels) > silhouette_pca_GMM_max:
        silhouette_pca_GMM_max = silhouette_score(rfm_pca, GMM_pca_labels)
        GMM_pca_cluster_opt = cluster_num
    
print(f"Optimal number of clusters: {GMM_pca_cluster_opt}")
print(f"Best silhouette score: {silhouette_pca_GMM_max:.3f}")

Optimal number of clusters: 3
Best silhouette score: 0.436


In [33]:
GMM_pca_silhouette_fig = px.line(
    x=range(2, 11),
    y=silhouette_pca_GMM_list
)
GMM_pca_silhouette_fig.update_layout(
        autosize=False, 
        width=1000,
        height=600,
        title_text="GMM, PCA, silhouette score vs number of clusters")
GMM_pca_silhouette_fig.layout.xaxis.title.text = "Number of clusters"
GMM_pca_silhouette_fig.layout.yaxis.title.text = "Silhouette score"
GMM_pca_silhouette_fig.show()

In [34]:
GMM_pca_opt = GaussianMixture(n_components=GMM_pca_cluster_opt, random_state=42)
GMM_pca_opt.fit(rfm_pca)
GMM_pca_labels_opt = GMM_pca_opt.predict(rfm_pca)

rfm_pca_GMM_fig = px.scatter(
    rfm_pca,
    x=rfm_pca[:, 0],
    y=rfm_pca[:, 1],
    color=GMM_pca_labels_opt
)
rfm_pca_GMM_fig.update_traces(marker_size=3)
rfm_pca_GMM_fig.update_layout(
    showlegend=False, 
    autosize=False, 
    width=1000,
    height=600,
    title_text=f"RFM, PCA, 2 components, Gaussian Mixture, {GMM_pca_cluster_opt} clusters")
rfm_pca_GMM_fig.show()

In [58]:
pca_table = pd.DataFrame({'Clustering type': ['Kmeans', 'Gaussian Mixture'],
                           'Optimal number of clusters': [kmeans_pca_cluster_opt, GMM_pca_cluster_opt],
                           'Maximum silhouette score': [silhouette_pca_kmeans_max, silhouette_pca_GMM_max]})
pca_table

,Clustering type,Optimal number of clusters,Maximum silhouette score
0,Kmeans,3,0.524230
1,Gaussian Mixture,3,0.436061


In [35]:
pd.Series(kmeans_pca_opt.labels_).value_counts()

2    2269
0     999
1     776
Name: count, dtype: int64

In [38]:
RFM_pca_labeled = RFM_filtered.copy()
RFM_pca_labeled['kmeans_label'] = kmeans_pca_opt.labels_
RFM_pca_labeled.groupby(by='kmeans_label').mean()

,Recency,Frequency,Monetary
kmeans_label,,,
0,253.474474,1.403403,386.277297
1,31.943299,7.068299,2657.746997
2,51.221684,2.329220,660.068947


In [37]:
def plot_cluster_profile(grouped_data, n_clusters):
    """Plot polar diagram of a clustered data.

    Args:
        grouped_data (pandas.DataFrame): clustered data to plot
        n_clusters (int): number of clusters
    """
    
    # Normalize data
    scaler = MinMaxScaler()
    grouped_data = pd.DataFrame(scaler.fit_transform(grouped_data), columns=grouped_data.columns)
    features = grouped_data.columns

    fig = go.Figure()
    for i in range(n_clusters):
        fig.add_trace(go.Scatterpolar(
            r=grouped_data.iloc[i].values,
            theta=features,
            fill='toself',
            name=f'Cluster {i}',
        ))
    fig.update_layout(
        showlegend=True,
        autosize=False,
        width=1000,
        height=1000,
    )
    fig.show()

In [40]:
plot_cluster_profile(RFM_pca_labeled.groupby('kmeans_label').mean(), kmeans_pca_cluster_opt)

## 5. RFM. t-SNE

In [41]:
tsne_pipeline = Pipeline([('scaler', StandardScaler()), 
                          ('t-SNE', TSNE(n_components=2, perplexity=50, random_state=100))])
rfm_tsne = tsne_pipeline.fit_transform(RFM_filtered)
print(f"KL divergence: {tsne_pipeline[1].kl_divergence_:.2f}")

KL divergence: 0.54


In [42]:
rfm_tsne_fig = px.scatter(
    rfm_tsne,
    x=rfm_tsne[:, 0],
    y=rfm_tsne[:, 1]
)
rfm_tsne_fig.update_traces(marker_size=3)
rfm_tsne_fig.update_layout(
    showlegend=False, 
    autosize=False, 
    width=1000,
    height=600,
    title_text="RFM, 2 components t-SNE applied")
rfm_tsne_fig.show()

### 5.1 Kmeans

In [43]:
silhouette_tsne_kmeans_max = 0
silhouette_tsne_kmeans_list = []
kmeans_tsne_cluster_opt = 0

for cluster_num in range(3, 9):
    kmeans_tsne_var = KMeans(n_clusters=cluster_num, random_state=42)
    kmeans_tsne_var.fit(rfm_tsne)
    silhouette_tsne_kmeans_list.append(silhouette_score(rfm_tsne, kmeans_tsne_var.labels_))
    if silhouette_score(rfm_tsne, kmeans_tsne_var.labels_) > silhouette_tsne_kmeans_max:
        silhouette_tsne_kmeans_max = silhouette_score(rfm_tsne, kmeans_tsne_var.labels_)
        kmeans_tsne_cluster_opt = cluster_num
    
print(f"Optimal number of clusters: {kmeans_tsne_cluster_opt}")
print(f"Best silhouette score: {silhouette_tsne_kmeans_max:.3f}")

Optimal number of clusters: 7
Best silhouette score: 0.484


In [44]:
kmeans_silhouette_tsne_fig = px.line(
    x=range(3, 9),
    y=silhouette_tsne_kmeans_list
)
kmeans_silhouette_tsne_fig.update_layout(
        autosize=False, 
        width=1000,
        height=600,
        title_text="Kmeans, t-SNE, silhouette score vs number of clusters")
kmeans_silhouette_tsne_fig.layout.xaxis.title.text = "Number of clusters"
kmeans_silhouette_tsne_fig.layout.yaxis.title.text = "Silhouette score"
kmeans_silhouette_tsne_fig.show()

In [45]:
kmeans_tsne_opt = KMeans(n_clusters=kmeans_tsne_cluster_opt, random_state=42)
kmeans_tsne_opt.fit(rfm_tsne)

rfm_tsne_kmeans_fig = px.scatter(
    rfm_tsne,
    x=rfm_tsne[:, 0],
    y=rfm_tsne[:, 1],
    color=kmeans_tsne_opt.labels_
)
rfm_tsne_kmeans_fig.update_traces(marker_size=3)
rfm_tsne_kmeans_fig.update_layout(
    showlegend=False, 
    autosize=False, 
    width=1000,
    height=600,
    title_text=f"RFM, t-SNE, Kmeans, {kmeans_tsne_cluster_opt} clusters")
rfm_tsne_kmeans_fig.show()

### 5.2 Gaussian Mixture

In [46]:
silhouette_tsne_GMM_max = 0
silhouette_tsne_GMM_list = []
GMM_tsne_cluster_opt = 0

for cluster_num in range(3, 9):
    GMM_tsne_var = GaussianMixture(n_components=cluster_num, random_state=42)
    GMM_tsne_var.fit(rfm_tsne)
    GMM_tsne_labels = GMM_tsne_var.predict(rfm_tsne)
    silhouette_tsne_GMM_list.append(silhouette_score(rfm_tsne, GMM_tsne_labels))
    if silhouette_score(rfm_tsne, GMM_tsne_labels) > silhouette_tsne_GMM_max:
        silhouette_tsne_GMM_max = silhouette_score(rfm_tsne, GMM_tsne_labels)
        GMM_tsne_cluster_opt = cluster_num
    
print(f"Optimal number of clusters: {GMM_tsne_cluster_opt}")
print(f"Best silhouette score: {silhouette_tsne_GMM_max:.3f}")

Optimal number of clusters: 4
Best silhouette score: 0.469


In [47]:
GMM_silhouette_tsne_fig = px.line(
    x=range(3, 9),
    y=silhouette_tsne_GMM_list
)
GMM_silhouette_tsne_fig.update_layout(
        autosize=False, 
        width=1000,
        height=600,
        title_text="GMM, t-SNE, silhouette score vs number of clusters")
GMM_silhouette_tsne_fig.layout.xaxis.title.text = "Number of clusters"
GMM_silhouette_tsne_fig.layout.yaxis.title.text = "Silhouette score"
GMM_silhouette_tsne_fig.show()

In [48]:
GMM_tsne_opt = GaussianMixture(n_components=GMM_tsne_cluster_opt, random_state=42)
GMM_tsne_opt.fit(rfm_tsne)
GMM_labels_tsne_opt = GMM_tsne_opt.predict(rfm_tsne)

rfm_tsne_GMM_fig = px.scatter(
    rfm_tsne,
    x=rfm_tsne[:, 0],
    y=rfm_tsne[:, 1],
    color=GMM_labels_tsne_opt
)
rfm_tsne_GMM_fig.update_traces(marker_size=3)
rfm_tsne_GMM_fig.update_layout(
    showlegend=False, 
    autosize=False, 
    width=1000,
    height=600,
    title_text=f"RFM, t-SNE, Gaussian Mixture, {GMM_tsne_cluster_opt} clusters")
rfm_tsne_GMM_fig.show()

### 5.3 Agglomerative clustering

In [49]:
silhouette_tsne_aggl_max = 0
silhouette_tsne_aggl_list = []
aggl_tsne_cluster_opt = 0

for cluster_num in range(2, 9):
    aggl_tsne_var = AgglomerativeClustering(n_clusters=cluster_num)
    aggl_tsne_var.fit(rfm_tsne)
    silhouette_tsne_aggl_list.append(silhouette_score(rfm_tsne, aggl_tsne_var.labels_))
    if silhouette_score(rfm_tsne, aggl_tsne_var.labels_) > silhouette_tsne_aggl_max:
        silhouette_tsne_aggl_max = silhouette_score(rfm_tsne, aggl_tsne_var.labels_)
        aggl_tsne_cluster_opt = cluster_num
    
print(f"Optimal number of clusters: {aggl_tsne_cluster_opt}")
print(f"Best silhouette score: {silhouette_tsne_aggl_max:.3f}")

Optimal number of clusters: 8
Best silhouette score: 0.479


In [50]:
aggl_silhouette_tsne_fig = px.line(
    x=range(2, 9),
    y=silhouette_tsne_aggl_list
)
aggl_silhouette_tsne_fig.update_layout(
        autosize=False, 
        width=1000,
        height=600,
        title_text="Agglomerative clustering, t-SNE, silhouette score vs number of clusters")
aggl_silhouette_tsne_fig.layout.xaxis.title.text = "Number of clusters"
aggl_silhouette_tsne_fig.layout.yaxis.title.text = "Silhouette score"
aggl_silhouette_tsne_fig.show()

In [52]:
aggl_tsne_opt = AgglomerativeClustering(n_clusters=aggl_tsne_cluster_opt)
aggl_tsne_opt.fit(rfm_tsne)

rfm_tsne_aggl_fig = px.scatter(
    rfm_tsne,
    x=rfm_tsne[:, 0],
    y=rfm_tsne[:, 1],
    color=aggl_tsne_opt.labels_
)
rfm_tsne_aggl_fig.update_traces(marker_size=3)
rfm_tsne_aggl_fig.update_layout(
    showlegend=False, 
    autosize=False, 
    width=1000,
    height=600,
    title_text=f"RFM, t-SNE, agglomerative clustering, {aggl_tsne_cluster_opt} clusters")
rfm_tsne_aggl_fig.show()

In [57]:
tsne_table = pd.DataFrame({'Clustering type': ['Kmeans', 'Gaussian Mixture', 'Agglomerative'],
                           'Optimal number of clusters': [kmeans_tsne_cluster_opt, GMM_tsne_cluster_opt, 
                                                          aggl_tsne_cluster_opt],
                           'Maximum silhouette score': [silhouette_tsne_kmeans_max, silhouette_tsne_GMM_max, 
                                                silhouette_tsne_aggl_max]})
tsne_table

,Clustering type,Optimal number of clusters,Maximum silhouette score
0,Kmeans,7,0.484495
1,Gaussian Mixture,4,0.468827
2,Agglomerative,8,0.479046


### 5.4 Profiling clusters

In [53]:
pd.Series(kmeans_tsne_opt.labels_).value_counts()

1    914
4    683
0    656
2    543
5    446
6    405
3    397
Name: count, dtype: int64

In [55]:
RFM_tsne_labeled = RFM_filtered.copy()
RFM_tsne_labeled['kmeans_label'] = kmeans_tsne_opt.labels_
RFM_tsne_labeled.groupby(by='kmeans_label').mean()

,Recency,Frequency,Monetary
kmeans_label,,,
0,46.329268,1.000000,321.538796
1,32.400438,6.770241,2416.582451
2,40.022099,1.979742,613.663941
3,313.000000,1.012594,261.105315
4,40.149341,3.450952,995.197350
5,195.800448,2.441704,670.317960
6,181.439506,1.000000,273.465160


In [56]:
plot_cluster_profile(RFM_tsne_labeled.groupby(by='kmeans_label').mean(), kmeans_tsne_cluster_opt)

## 6. Prediction

In [63]:
X = RFM_filtered
y = kmeans_tsne_opt.labels_
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test.shape[0]

809

### 6.1 Random Forest Classifier

In [65]:
gs_rfc = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                      param_grid={'max_depth': range(5, 15), 
                                  'criterion': ['gini', 'entropy'], 
                                  'n_estimators': [100, 200, 500]},
                      cv=5,
                      scoring='accuracy',
                      n_jobs=-1
                      )
gs_rfc.fit(X_train, y_train)
print(f"Random Forest Classifier, optimal parameters: {gs_rfc.best_params_}")

Optimal parameters: {'criterion': 'gini', 'max_depth': 13, 'n_estimators': 200}


In [68]:
rfc_opt = RandomForestClassifier(**gs_rfc.best_params_, random_state=42)
rfc_opt.fit(X_train, y_train)
y_pred_rfc_test = rfc_opt.predict(X_test)
print(f"Random Forest Classifier, test sample, accuracy score: {accuracy_score(y_pred_rfc_test, y_test):.3f}")

Accuracy score, test sample: 0.985


### 6.2 Gradient Boosting Classifier

In [70]:
gs_gbc = GridSearchCV(estimator=GradientBoostingClassifier(random_state=42),
                      param_grid={'max_depth': range(3, 7), 
                                  'learning_rate': [0.001, 0.01, 0.1],
                                  'n_estimators': [100, 200, 500]},
                      cv=5,
                      scoring='accuracy',
                      n_jobs=-1
)
gs_gbc.fit(X_train, y_train)
print(f"Gradient Boosting Classifier, optimal parameters: {gs_gbc.best_params_}")

Gradient Boosting Classifier, optimal parameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200}


In [71]:
gbc_opt = GradientBoostingClassifier(**gs_gbc.best_params_, random_state=42)
gbc_opt.fit(X_train, y_train)
y_pred_gbc_test = gbc_opt.predict(X_test)
print(f"Gradient Boosting Classifier, test sample, accuracy score: {accuracy_score(y_pred_gbc_test, y_test):.3f}")

Gradient Boosting Classifier, test sample, accuracy score: 0.983
